In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from __future__ import print_function
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, CuDNN 4007)


In [2]:
class CharacterTable(object):
    def __init__(self, vocab, maxlen):
        self.vocab = vocab
        self.maxlen = maxlen
    
    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.vocab)))
        for i, c in enumerate(C):
            X[i, c] = 1
        return X
    
    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ','.join(x for x in X)

In [3]:
def generateRandSeq(min, max, len):
    return [np.random.randint(min, max) for _ in range(len)]

In [46]:
TRAINING_SIZE = 150000
TEST_SIZE = 10000
DIGITS = 25
MAXLEN = DIGITS
voc = list(xrange(100))
ctable = CharacterTable(voc, MAXLEN)

In [47]:
inputs = []
outputs = []
inputs_t = []
outputs_t = []
print('Generating data...')
while len(inputs) < TRAINING_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs.append(s)
    outputs.append(s[::-1])

while len(inputs_t) < TEST_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs_t.append(s)
    outputs_t.append(s[::-1])

print(inputs[12])
print(outputs[12])

Generating data...
[3, 89, 93, 6, 7, 79, 78, 14, 87, 90, 10, 14, 55, 23, 61, 44, 18, 63, 54, 49, 40, 60, 45, 36, 43]
[43, 36, 45, 60, 40, 49, 54, 63, 18, 44, 61, 23, 55, 14, 10, 90, 87, 14, 78, 79, 7, 6, 93, 89, 3]


In [48]:
print('Vectorization...')
X = np.zeros((len(inputs), MAXLEN, len(voc)), dtype=np.bool)
y = np.zeros((len(outputs), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)

for i, sentence in enumerate(outputs):
    y[i] = ctable.encode(sentence, maxlen=MAXLEN)

X_test = np.zeros((len(inputs_t), MAXLEN, len(voc)), dtype=np.bool)
y_test = np.zeros((len(outputs_t), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs_t):
    X_test[i] = ctable.encode(sentence, maxlen=MAXLEN)

for i, sentence in enumerate(outputs_t):
    y_test[i] = ctable.encode(sentence, maxlen=MAXLEN)

Vectorization...


In [49]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(150000, 25, 100)
(150000, 25, 100)
(10000, 25, 100)
(10000, 25, 100)


In [50]:
HIDDEN_SIZE = 128
BATCH_SIZE = 200
LAYERS = 2

In [51]:
print('Build model...')
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(voc)), return_sequences=True))
for _ in range(LAYERS - 2):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(len(voc)))
model.add(Activation('softmax'))

model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [52]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
hist = model.fit(X, y, batch_size=BATCH_SIZE, nb_epoch=200, 
#                  callbacks=[early_stopping],
          validation_split = 0.1, shuffle=True)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 135000 samples, validate on 15000 samples
Epoch 1/200
135000/135000 [==============================] - 422s - loss: 4.0046 - acc: 0.0845 - val_loss: 3.6689 - val_acc: 0.1237
Epoch 2/200
135000/135000 [==============================] - 549s - loss: 3.4274 - acc: 0.1530 - val_loss: 3.2332 - val_acc: 0.1768
Epoch 3/200
135000/135000 [==============================] - 550s - loss: 3.0866 - acc: 0.1944 - val_loss: 2.9661 - val_acc: 0.2093
Epoch 4/200
135000/135000 [==============================] - 459s - loss: 2.8553 - acc: 0.2219 - val_loss: 2.7624 - val_acc: 0.2333
Epoch 5/200
135000/135000 [==============================] - 431s - loss: 2.6727 - acc: 0.2434 - val_loss: 2.6054 - val_acc: 0.2528
Epoch 6/200
135000/135000 [==============================] - 432s - loss: 2.5117 - acc: 0.2652 - val_loss: 2.4354 - val_acc: 0.2774
Epoch 7/200
135000/135000 [==============================] - 433s - loss: 2.3620 - acc: 0.2889 - val_loss: 2.3115 - val_acc: 0.2990
Epoch 8/200
135000/135000

In [124]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

10000/10000 [==============================] - 10s    
Test score: 0.160358118713
Test accuracy: 0.942218


In [70]:
test = '12345678901234567898'
x_test = ctable.encode(test, maxlen=MAXLEN)
X_test = np.zeros((1, MAXLEN, len(chars)), dtype=np.bool)
X_test[0] = x_test
res = model.predict_classes(X_test)
res[0]

1/1 [==============================] - 0s


array([8, 8, 7, 7, 6, 5, 4, 3, 2, 1, 0, 0, 8, 7, 7, 5, 4, 3, 2, 1])

In [28]:
import matplotlib.pyplot as plt
axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['loss'], color='g')
plt.plot(hist.history['loss'], color='g', label='Training Loss')
plt.scatter(hist.epoch, hist.history['val_loss'], color='b')
plt.plot(hist.history['val_loss'], color='b', label='Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.title('Training Loss & Validation Loss vs Epochs')
plt.legend()

plt.figure(2)

axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['acc'], color='r')
plt.plot(hist.history['acc'], color='r', label='Training Accuracy')
plt.scatter(hist.epoch, hist.history['val_acc'], color='c')
plt.plot(hist.history['val_acc'], color='c', label='Validation Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Trainging Accuracy & Validation Accuracy vs Epochs')
plt.legend()

plt.show()